In [1]:
import nltk
import re
import pandas as pd

# Download necessary NLTK resources
nltk.download('wordnet')


def remove_symbols(text: str) -> str:
    """remove punctuation, symbols and etc."""

    fix = text
    fix = re.sub(r"\s+", " ", fix)
    fix = re.sub(r"\d+", " ", fix)
    fix = re.sub(r"([.!?])", r" ", fix)
    fix = re.sub(r"[^a-zA-Z.!?]+", r" ", fix)
    fix = fix.strip()
    fix = fix.lower()

    return fix

[nltk_data] Downloading package wordnet to /home/cosmos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import pandas as pd
import numpy as np
# data_path = '/content/drive/MyDrive/filtered.tsv'
# model_cktp_path = 'pretrained.pt'

# data = pd.read_csv(
#     data_path, sep="\t", index_col=0
# )
# data.head()

In [5]:
import os
os.getcwd()

'/home/cosmos/VScode Projects/MiraiVision/MLandDL/detoxification/solution/notebooks'

In [15]:
negative_words = pd.read_csv("../data/raw/negative-words.txt", header=None)
toxic_words = pd.read_csv("../data/raw/toxic_words.txt", header=None)


toxic_df = toxic_words.append(negative_words, ignore_index=True)
toxic_df.rename(columns={ toxic_df.columns[0]: "tox" }, inplace = True)
toxic_df.head()

,tox
0,4r5e
1,5h1t
2,5hit
3,a55
4,anal


In [9]:
from nltk.corpus import wordnet
def untoxify_word(word, blacklist):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            # Check against blacklist
            if lemma.name().lower() not in blacklist:
                synonyms.add(lemma.name())

    return list(synonyms)[0] if synonyms else None

In [17]:
# Using the first column as a blacklist
blacklist = set(toxic_df["tox"].str.lower())

# Apply the function to get non-toxic synonyms
toxic_df["ntox"] = toxic_df["tox"].apply(lambda x: untoxify_word(x, blacklist))
toxic_df = toxic_df.dropna()  # some of the results are None, so drop them

In [18]:
toxic_df.head()

,tox,ntox
4,anal,anal_retentive
8,arse,nates
9,ass,nates
11,asses,nates
14,asshole,bunghole


In [24]:

# Create a replacement dictionary
replacement_dict = dict(zip(toxic_df["tox"], toxic_df["ntox"]))


# Function to replace toxic words in a sentence
def replace_toxic_words(sentence):
    for toxic, non_toxic in replacement_dict.items():
        # \b specifies word boundaries in regex, ensuring we're replacing whole words, not substrings
        sentence = re.sub(r"\b" + re.escape(toxic) + r"\b", non_toxic, sentence)
    return sentence


replace_toxic_words("You are a nasty bastard!")

'You are a tight illegitimate_child!'